In [1]:
% matplotlib 

Using matplotlib backend: Qt4Agg


In [2]:
% matplotlib inline

In [3]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import datetime
import time
from datetime import datetime, date

In [4]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import  SVC
from sklearn.metrics import accuracy_score, classification_report

from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import  SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
#from sklearn.feature_extraction import DictVectorizer
#from sklearn.grid_search import GridSearchCV 

### 讀取csv檔：

In [5]:
dataset = pd.read_csv('1127.csv')

In [7]:
c320 = 19.05
dataset['R'] = (c320**2 + dataset['x']**2)/(2*dataset['x'])

In [6]:
#dataset.columns

In [8]:
dataset.head(1)

,x,R,T1,T2,T3,T4,T5,speed,env_T,env_hum,set_T1,set_T2,set_T3,set_T4,set_T5,piece,position,time,ok
0,0.572,317.508465,580,736,722,621,488,32.34,29,13,750,735,722,621,500,1,1,2017/11/27 11:45,1


### 選擇任務範圍:

In [9]:
dataset = dataset.drop(['x','set_T1', 'set_T2', 'set_T3', 'set_T4', 'set_T5'
                       ], axis = 1)

In [10]:
#dataset.columns

In [11]:
dataset.head(1)

,R,T1,T2,T3,T4,T5,speed,env_T,env_hum,piece,position,time,ok
0,317.508465,580,736,722,621,488,32.34,29,13,1,1,2017/11/27 11:45,1


In [12]:
len(dataset)

336

### 選取ok資料做回歸 限制輸送帶速度

In [15]:
data_ok = dataset[dataset['ok']==1]
data_ok = data_ok[data_ok['speed']<30 ]

In [21]:
data_ok.describe()

,R,T1,T2,T3,T4,T5,speed,env_T,env_hum,piece,position,ok
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,1.200000e+02,120.000000,120.000000,120.000000,120.000000,120.0
mean,325.188492,626.000000,722.250000,712.250000,614.000000,450.450000,2.238000e+01,26.300000,14.300000,3.000000,3.500000,1.0
std,7.706089,12.875617,19.018456,16.958303,12.175192,5.658117,6.778459e-14,0.460179,0.460179,2.289912,1.714986,0.0
min,311.528636,598.000000,690.000000,683.000000,593.000000,441.000000,2.238000e+01,26.000000,14.000000,1.000000,1.000000,1.0
25%,319.179491,620.000000,716.250000,712.250000,614.000000,448.500000,2.238000e+01,26.000000,14.000000,1.750000,2.000000,1.0
50%,323.723013,632.000000,735.000000,722.000000,621.000000,453.000000,2.238000e+01,26.000000,14.000000,2.000000,3.500000,1.0
75%,330.186364,636.000000,735.000000,722.000000,621.000000,454.000000,2.238000e+01,27.000000,15.000000,3.000000,5.000000,1.0
max,343.921670,636.000000,735.000000,722.000000,621.000000,456.000000,2.238000e+01,27.000000,15.000000,9.000000,6.000000,1.0


### 線性回歸器

In [16]:
X_train, X_test, y_train, y_test = train_test_split(data_ok[data_ok.columns[1:9]],
                     data_ok[data_ok.columns[0]],test_size=0.0,random_state=123)

In [17]:
#ss = StandardScaler()
#X_train = ss.fit_transform(X_train)
#X_test = ss.fit_transform(X_test)

In [18]:
lr = LinearRegression(n_jobs=-1)
lr.fit(X_train, y_train)
#lr_predict = lr.predict(X_test)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [19]:
lr_train = lr.predict(X_train)

In [20]:
r2_score(y_train, lr_train) 

0.035542204639614328

### 驗證R2 score定義

In [21]:
mean = np.mean(data_ok['R'])
AA = (y_train-lr_train)
BB = (y_train-mean)
1-np.mean(AA**2)/np.mean(BB**2)

0.035542204639614994

### Create a new dataframe:

In [22]:
#columns = ['R', 'T1', 'T2', 'T3', 'T4', 'T5', 'speed', 'env_T', 'env_hum', 'piece',
#       'position', 'ok']
#df_save = pd.DataFrame(np.array(np.zeros([120,12])), columns=columns)
#df_save.index = list(range(120))

In [23]:
#CC = AA-BB
#df_save = pd.concat([ AA,BB,CC], axis=1)
#df_save.to_csv('df_save.csv')

In [98]:
#print(X_train[abs(AA)>15].sort_index())
#CC[abs(CC)>100].sort_values()
#print(len(X_train[abs(AA)>15]))

### 線性回歸的權重

In [42]:
print(lr.coef_)
print(lr.intercept_)

[-0.00356847 -0.47913636  0.00328417  0.00235787  1.50548679  0.
  1.11918378  1.62381896]
-61.1099562863


### 測試線性回歸器的參數預測方法

In [42]:
X_present = [[632, 690, 683, 593, 441, 22.38, 26, 14]]
R_present = lr.predict(X_present)
X_test  = [[633, 691, 684, 594, 442, 22.38, 27, 15]]
R_test  = lr.predict(X_test)
print(R_present, R_test)

(array([ 325.42390119]), array([ 329.19532792]))


In [60]:
A_2 = sum(np.square(lr.coef_))
delX = (lr.coef_/A_2)*( R_test - R_present)

In [48]:
X_output= X_input + delX
R_output= lr.predict(X_output)
print(X_output - X_test)
print(R_output - R_test)

[[-1.00210764 -1.28299137 -0.99806028 -0.99860738 -0.11081729  0.
  -0.33897868 -0.04092699]]
[ -5.68434189e-14]


### 線性回歸器的參數預測

In [74]:
def parameter_pred_linear(regressor, X_present, R_target):
    ''' The regressor should be pretrained. '''
    R_present = regressor.predict([X_present])
    A_2 = sum(np.square(regressor.coef_))
    delX = (regressor.coef_/A_2)*( R_target - R_present)
    X_output= X_input + delX
#    R_output= regressor.predict(X_output)
    return R_present, X_output

In [72]:
# An example:
X_present = [632, 690, 683, 593, 441, 22.38, 26, 14]
R_target = 329.2
R_present, X_output = parameter_pred_linear(lr, X_present, R_target)

### 任意回歸器的參數預測

In [82]:
## type(X) should be a list (a single vector)
def gradient(regressor, X):
    X_ = np.array(X)
    dim_X = X_.shape[1]
    u = np.zeros(dim_X)
    grad = np.zeros(dim_X)
    eps = 1e-5
    f0 = regressor.predict(X_)
    for i in range(dim_X):
        u[i]=1
        f1 = regressor.predict(X_+eps*u)
        grad[i] = (f1-f0)/eps
        u[i]=0
#    for i in np.nonzero(constraint):
#        u[i]=1
#        f1 = regressor.predict(X_+eps*u)
#        grad[i] = (f1-f0)/eps
#        u[i]=0
    return grad

In [91]:
def parameter_pred(regressor, X_present, f_target, constraint, steps=100):
    ''' 1. The regressor should be pretrained. 
        2. constraint is a list defining which components of X_present are fixed:
            [1, 1, 0, 1] corresponds to one fixed (3rd)component.
    '''
    f_present = regressor.predict(X_present)[0]
    del_f = (f_target - f_present)/steps
    X_ = np.array(X_present)
    P_ = np.array(constraint)
    f_ = f_present
    for i in range(steps):
        partial_grad = gradient(regressor, X_)*P_
        X_ += partial_grad*del_f/sum(np.square(partial_grad))
        f_ += del_f
    return [ f_present, X_, f_,]

In [96]:
# An example:
constraint = [1,1,1,1,1,1,0,0]
R_present, X_output, R_output = parameter_pred(lr, X_input, 340.19532792, constraint)

In [97]:
print(X_input)
print(R_present)
print(X_output)
print(R_output)

[[632, 690, 683, 593, 441, 22.38, 26, 14]]
325.423901192
[[ 631.97888243  687.16455564  683.01943514  593.01395343  449.90920486
    22.38         26.           14.        ]]
340.19532792


### 多項式回歸器

In [361]:
X_train, X_test, y_train, y_test = train_test_split(data_ok[data_ok.columns[1:9]],
                     data_ok[data_ok.columns[0]],test_size=0.0,random_state=123)

In [362]:
(X_train).shape

(120, 8)

In [367]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3)
X_train_poly = poly.fit_transform(X_train)

In [368]:
X_train_poly.shape

(120, 165)

In [369]:
from sklearn.pipeline import Pipeline
model = Pipeline([('poly', PolynomialFeatures(degree=4)),
                 ('linear', LinearRegression(fit_intercept=False))])
# fit to an order-3 polynomial data
model = model.fit(X_train, y_train)
# model.named_steps['linear'].coef_
poly_train = model.predict(X_train)

In [370]:
print( r2_score(y_train, poly_train)  )

0.0355422046396


In [37]:
print('LR default measure:', lr.score(X_train, y_train))

('LR default measure:', 0.035542204639614328)


In [ ]:
print('LR default measure:', lr.score(X_test, y_test))

### 隨機森林回歸器

In [352]:
X_train, X_test, y_train, y_test = train_test_split(data_ok[data_ok.columns[1:9]],
                     data_ok[data_ok.columns[0]],test_size=0.25,random_state=22)

In [353]:
rfr = RandomForestRegressor( bootstrap= False ,n_jobs = -1, n_estimators=100,
                               max_features='sqrt')
rfr.fit(X_train, y_train)
rfr_predict = rfr.predict(X_test)

In [354]:
rfr_train = rfr.predict(X_train)

In [355]:
print( r2_score(y_test, rfr_predict)  )

-0.313620678162


In [356]:
print( r2_score(y_train, rfr_train)  )

0.0798935713542


### SVR 回歸任務

In [357]:
X_train, X_test, y_train, y_test = train_test_split(data_ok[data_ok.columns[1:9]],
                     data_ok[data_ok.columns[0]],test_size=0.0,random_state=22)
#ss = StandardScaler()
#X_train = ss.fit_transform(X_train)
#X_test = ss.fit_transform(X_test)

In [358]:
svr = SVR( kernel = 'rbf', C = 2.0, epsilon= 1.0,  gamma = 0.1)
svr.fit(X_train, y_train)
#SVR(C=2.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.09,
#  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

SVR(C=2.0, cache_size=200, coef0=0.0, degree=3, epsilon=1.0, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [360]:
svr_train = svr.predict(X_train)
r2_score(y_train, svr_train )

-0.027289218586723507

In [ ]:
svr_predict = svr.predict(X_test)
r2_score(y_test, svr_predict )

### 切分訓練/測試資料集：

In [10]:
X_train, X_test, y_train, y_test = train_test_split(dataset[dataset.columns[0:8]],
                     dataset[dataset.columns[9]],test_size=0.2,random_state=22)

In [11]:
print([X_train.shape, X_test.shape, y_train.shape, y_test.shape] )
print(y_train.value_counts())
print(y_test.value_counts())

[(1980, 8), (660, 8), (1980L,), (660L,)]
2    1076
0     458
1     446
Name: category, dtype: int64
2    364
1    154
0    142
Name: category, dtype: int64


### 前處理：

In [12]:
ss = StandardScaler()

In [13]:
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

### 1. 分類任務
### 1.1  以logistic regression作基準：

In [14]:
lr = LogisticRegression(n_jobs=-1)

In [15]:
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
lr_y_predict = lr.predict(X_test)

In [17]:
#lr_y_predict

In [18]:
print ('LR accuracy:', lr.score(X_test, y_test))

('LR accuracy:', 1.0)


### 1.2 測試SVM:

In [43]:
svc = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto',
          coef0=0.0, shrinking=True, probability=False,
          tol=0.001, cache_size=200, class_weight=None,
          verbose=False, max_iter=-1, decision_function_shape='ovr',
          random_state=None)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(dataset[dataset.columns[0:8]],
                     dataset[dataset.columns[9]],test_size=0.25,random_state=22)
ss_X = StandardScaler()
#ss_y = StandardScaler()
X_train = ss_X.fit_transform(X_train)
X_test = ss_X.transform(X_test)

In [45]:
svc.fit(X_train, y_train)
svc_val_predict = svc.predict(X_test)
#svc_val_predict
print(accuracy_score(y_test, svc_val_predict))

1.0


### 2. 聚類任務
### 2.1 用K-means聚類

In [19]:
X_train, X_test, y_train, y_test = train_test_split(dataset[dataset.columns[0:8]],
                     dataset[dataset.columns[9]],test_size=0.25,random_state=23)

In [20]:
from sklearn.cluster import KMeans

In [21]:
kmeans = KMeans(n_clusters = 5, n_jobs=-1)

In [22]:
kmeans.fit(X_train)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=5, n_init=10,
    n_jobs=-1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [23]:
y_pred = kmeans.predict(X_test)

In [24]:
df_y_pred = pd.DataFrame(y_pred)
df_y_pred.columns = ['digit']
df_y_pred.digit.value_counts()

0    371
3    146
2    101
4     37
1      5
Name: digit, dtype: int64

### 用adjusted rand index(ARI)評估聚類結果：

In [25]:
from sklearn import metrics
print (metrics.adjusted_rand_score(y_test, y_pred)  )
#用輪廓係數(silhouette coefficient, sc)來評估聚類效果：
#須定義凝聚度(cohesion)=a 和 分離度(separation)=b
#sc[i] = (b[i]-a[i])/max(b[i],a[i])
# 0<sc<1: 聚類效果好
#-1<sc<0: 聚類效果差

0.946617722774


### 用silhouette_score度量最適合聚類數目：

In [26]:
from sklearn.metrics import silhouette_score

In [27]:
clusters = [2,3,4,5,8]
sc_scores = []
for t in clusters:
    kmeans_model = KMeans(n_clusters = t).fit(X_test)
    sc_score = silhouette_score(X_test, kmeans_model.labels_, metric='euclidean')
    sc_scores.append(sc_score)
plt.figure()
plt.plot(clusters, sc_scores, '*-')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Coefficient Score')
plt.show()
#np.argmax(sc_scores)

#### 對於不同物理量構成的資料，"距離"觀念其實是有問題的（相同溫度差，濕度差，和速率差的無法比較），所以K-means不適用。

### 2.2 用t-SNE看聚類：

In [29]:
X_train, X_test, y_train, y_test = train_test_split(dataset[dataset.columns[0:8]],
                     dataset[dataset.columns[9]],test_size=0.25,random_state=22)
ss_X = StandardScaler()
#ss_y = StandardScaler()
X_train = ss_X.fit_transform(X_train)
X_test = ss_X.transform(X_test)

In [30]:
from sklearn.manifold import TSNE
time_start = time.time()
tsne = TSNE(n_components=3, verbose=1, perplexity=50, n_iter=300)
tsne_results = tsne.fit_transform(X_train)

print 't-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start)

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 1980
[t-SNE] Computed conditional probabilities for sample 1980 / 1980
[t-SNE] Mean sigma: 0.000000
[t-SNE] Error after 100 iterations with early exaggeration: 0.585949
[t-SNE] Error after 200 iterations: 0.588272
t-SNE done! Time elapsed: 32.4020001888 seconds


In [31]:
colors = y_train
c = plt.scatter(tsne_results[:,0], tsne_results[:,1], c=colors, cmap=plt.cm.RdYlGn)

### 3D視覺化

In [35]:
from mpl_toolkits.mplot3d import Axes3D as ax
import random

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(tsne_results[:,0],tsne_results[:,1], tsne_results[:,2], s=30, c=y_train)
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()